# Practice 1: Syntax-directed translation

Design a grammar using a bottom - up translator that accepts the following input in C language:
    - Assignment statement
    - Conditional instructions (if - else)
    - Comparison operators (==, <=, >=, !=)
    - Logic operators (AND, OR, NOT)
    - Arithmetic operators (+, - , *, / and unary operator (-))
    - Variables, numerical constants and vectors

In [ ]:
from sly import Lexer, Parser
import sys

tabla={}

class NodoPrint():
    def P(a):
        if a in tabla:
            print(tabla[a])
        else:
            print("Syntax error: ID '%c' isnt in table." %a) 

class NodoID():
    def P(a,b):
        tabla[a]=b
        print(tabla)

class NodoGen():
    def P(a):
        return a    
    
class NodoMenor():
    def P(a,b):
        return a<b

class NodoMayor():
    def P(a,b):
        return a>b
    
class NodoMenorIg():
    def P(a,b):
        return a<=b

class NodoMayorIg():
    def P(a,b):
        return a>=b

class NodoIg():
    def P(a,b):
        return a==b
    
class NodoAnd():
    def P(a,b):
        return a&b

class NodoOr():
    def P(a,b):
        return a|b

class NodoNot():
    def P(a):
        return ~a
    
class NodoSum():
    def P(a,b):
        return a+b
    
class NodoResta():
    def P(a,b):
        return a-b
    
class NodoMul():
    def P(a,b):
        return a*b
    
class NodoDiv():
    def P(a,b):
        return a/b
    
class NodoRU():
    def P(a):
        return -a
    
class NodoTabla():
    def P(a):
        return tabla[a]

class CalcLexer(Lexer):
    tokens = {DIGIT, ID, LESEQ, BIGEQ, EQ, PRINTF, PGENERIC, IF } #, ELSE}
    ignore = ' \t'
    DIGIT = r'[0-9]+'
    PRINTF = r'printf'
    PGENERIC = r'%d'
    IF = r'if'
    #ELSE = r'else'
    ID = r'[a-zA-Z][a-zA-Z0-9]*'
    
    LESEQ = r'<='
    BIGEQ = r'>='
    EQ = r'=='
    
    literals = {'+', '-', '*', '/', '(', ')', '<', '>', '=', ';', '|', '&', '~', ',', '"', '{', '}'}
    
    @_(r'[0-9]+')
    def DIGIT(self, f):
        f.value=int(f.value)
        return f
    
    def ID(self, f):
        f.value = str(f.value)
        return f
    
    def error(self, t):
        print('Bad character %r' % t.value[0])
        sys.exit(0)    
    
class CalcParser(Parser):
    tokens=CalcLexer.tokens
    def __init__(self):
        self.names={}
    
    @_('')
    def entrada(self, p):
        pass
    
    @_('entrada datos')
    def entrada(self,p):
        pass
    
    @_('entrada sentenciaIF')
    def entrada(self, p):
        pass
    
    @_('IF "(" expr ")" "{" entradaIF "}"')
    def sentenciaIF(self, p):
        pass 

    @_('entradaIF datosIF')
    def entradaIF(self,p):
        pass
    
    @_('')
    def entradaIF(self, p):
        pass

    @_('PRINTF "(" """ PGENERIC mas ID ")" ";"')
    def datosIF(self, p):
        if p[-2]:
            NodoPrint.P(p.ID)
        
    @_('ID "=" expr ";"')
    def datosIF(self, p):
        #Intento buscar el valor de expr de la regla anterior sentenciaIF -> IF ( expr ) { entradaIF }.
        #No tengo ni idea de dónde encon
        print(p[-1])
        print(p[-2])
        print(p[-3])
        print(p[-4])
        if p[-2]:
            NodoID.P(p.ID, p.expr)
    
   # @_('IF "(" expr ")" "{" datos "}" ELSE "{" datos1 "}" entrada')
    #def masdatos(self, p):
     #   if(True):
      #      return p.datos
       # else:
        #    return p.datos1
    
    
    @_('PRINTF "(" """ PGENERIC mas ID ")" ";"')
    def datos(self, p):
        NodoPrint.P(p.ID)
            
    @_('""" ","')
    def mas(self,p):
        pass
    
    @_('PGENERIC mas ID ","')
    def mas(self ,p):
        NodoPrint.P(p.ID)        
        
    @_('ID "=" expr ";"')
    def datos(self, p):
        NodoID.P(p.ID, p.expr)

    @_('expr "<" comp')
    def expr(self, p):
        return NodoMenor.P(p.expr, p.comp)
    
    @_('expr ">" comp')
    def expr(self, p):
        return NodeoMayor.P(p.expr, p.comp)
    
    @_('expr LESEQ comp')
    def expr(self, p):
        return NodoMenorIg.P(p.expr, p.comp)
    
    @_('expr BIGEQ comp')
    def expr(self, p):
        return NodeoMayorIg.P(p.expr, p.comp)
    
    @_('expr EQ comp')
    def expr(self, p):
        return NodoIg.P(p.expr, p.comp)
    
    @_('expr "&" comp')
    def expr(self, p):
        return NodoAnd.P(p.expr, p.comp)
    
    @_('expr "|" comp')
    def expr(self, p):
        return NodoOr.P(p.expr, p.comp)
    
    @_('"~" expr')
    def expr(self, p):
        return NodoNot.P(p.expr)
    
    @_('comp')
    def expr(self, p):
        return NodoGen.P(p.comp)
    
    @_('comp "+" sum')
    def comp(self, p):
        return NodoSum.P(p.comp, p.sum)
    
    @_('comp "-" sum')
    def comp(self, p):
        return NodoResta.P(p.comp, p.sum)
    
    @_('sum')
    def comp(self, p):
        return NodoGen.P(p.sum)
    
    @_('sum "*" fact')
    def sum(self, p):
        return NodoMul.P(p.sum, p.fact)
    
    @_('sum "/" fact')
    def sum(self, p):
        return NodoDiv.P(p.sum, p.fact)
    
    @_('fact')
    def sum(self, p):
        return NodoGen.P(p.fact)
    
    @_('"-" fact')
    def fact(self, p):
        return NodoRU.P(p.fact)
    
    @_('ID')
    def fact(self, p):
        return NodoTabla.P(p.ID)
    
    @_('"(" expr ")"')
    def fact(self, p):
        return NodoGen.P(p.expr)
    
    @_('DIGIT')
    def fact(self, p):
        return NodoGen.P(p.DIGIT)

if __name__=='__main__':
    lexer=CalcLexer()
    parser=CalcParser()
    while True:
        try:
            text=input('one expression plix > ')
        except EOFError:
            break
        if text:
            parser.parse(lexer.tokenize(text))
        

one expression plix > x=3+4;
{'x': 7}
one expression plix > printf("%d", x);
7


#####  José Luis Cumbrera Sánchez, Teodoro Martínez Márquez